In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

#mira is beautiful

In [ ]:
experiments = pd.read_csv('data/policy-experiments.csv')
outcomes = pd.read_csv('data/policy-outcomes.csv') #maybe outcomes.to_numpy() necessary

## Visualisations

In [ ]:
#For each of the outcomes classes (e.g. total cost, casualities) plot the range of outcome values generated under the sampled uncertainty scenarios
from ema_workbench.analysis import plotting, plotting_util

for outcome in outcomes.keys():
    plotting.lines(experiments, outcomes, outcomes_to_show=outcome,
                   density=plotting_util.Density.HIST)
plt.show()

In [ ]:
#Simple Visual Subspace partitioning: IF only 2 uncertainties present, shows visually the area where highest/lowest values of 1 outcome are found

# Calculate the maximum value of outcome I
I = outcomes['I']
maxI = np.max(I, axis=1)
# Calculate the timing of the maximum value of I
timing = np.argmax(I, axis=1)

def plot_scatter(labels=True, beta_is_gamma_line=True):
    # Plot the relationship between beta and gamma, using max I and timing as color
    fig, ax = plt.subplots(1,2, figsize=(12,6))

    for i, hue in enumerate([maxI, timing]):
        sc = ax[i].scatter(experiments.beta, experiments.gamma, c=hue, cmap='viridis')
        cb = plt.colorbar(sc)
        ax[i].set_title(f"Relationship between beta, gamma and {['max. I', 'timing of max I'][i]}")
        ax[i].set_xlabel('Beta (contact rate)')
        ax[i].set_ylabel('Gamma (recovery rate)')
        cb.set_label(['max. infections', 'time step of max. I.'][i])

        # Add labels with the hue value
        if labels:
            for j, (beta, gamma) in enumerate(zip(experiments.beta, experiments.gamma)):
                ax[i].text(beta, gamma, f"{hue[j]:.0f}", fontsize=8)

        # To the second plot, add a line with beta = gamma
        if i == 1 and beta_is_gamma_line:
            ax[i].plot(np.linspace(0.05,0.15), np.linspace(0.05,0.15), color='red')
    plt.show()

plot_scatter()


In [ ]:
#Simple Visual Correlation Analysis: for 2 outcomes of interest

# Calculate the maximum value of outcome I
I = outcomes['I']
maxI = np.max(I, axis=1)
# Calculate the timing of the maximum value of I
timing = np.argmax(I, axis=1)

fig, ax = plt.subplots(figsize=(6,6))

sc = ax.scatter(timing, maxI)
ax.set_title('Timing and severity of outbreak')
ax.set_xlabel('Point in time')
ax.set_ylabel('Max. I (infected)')
plt.show()

In [ ]:
#Simple Visual Correlation and Trade-Off Analysis: for multiple outcomes of interest; Which uncertainties generate system behaviour?
from ema_workbench.analysis import pairs_plotting

pairs_plotting.pairs_scatter(experiments, outcomes)

fig = plt.gcf()
fig.set_size_inches(8,8)

plt.show()

#alternative seaborn sns based plot
experiments, outcomes = results
policies = experiments['policy']

data = pd.DataFrame.from_dict(outcomes)
data['policy'] = policies

sns.pairplot(data, hue='policy',  vars=outcomes.keys(), )
plt.show()

## Analysis

In [ ]:
#Scenario Discovery PRIM, should not be used if many uncertainty parameters and very non-linear behaviour, only for 1 outcome
from ema_workbench.analysis import prim
from ema_workbench import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO)

prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha=0.1) #replace x with experiments, y with outcomes
box1 = prim_alg.find_box()

box1.show_tradeoff(annotated=True)
plt.show()

point=10
box1.inspect(point, style='graph')
plt.show()

box1.resample(point) #those with high percentages are reliable under resampling, those with very low % can be ignored in interpretation (bc. not reliable)

box1.select(21)
box1.show_pairs_scatter()
fig = plt.gcf()
fig.set_size_inches(12,12)
plt.show()

In [ ]:
#Subspace partitioning for 1 outcome and 2 uncertainties (same as PRIM but with restricted set of acceptable outcomes)
data = outcomes['reliability']

y = data < np.percentile(data, 10) #restricting set of outcomes

prim_alg = prim.Prim(cleaned_experiments,y, threshold=0.8)
box1 = prim_alg.find_box()

from ema_workbench.analysis import dimensional_stacking
dimensional_stacking.create_pivot_plot(cleaned_experiments, y) #plot should not contain white spaces (otherwise generate more scenarios)

#Why do we get this box? What is the structural explanation for it?